* Master DAC - BDLE
* Author: Mohamed-Amine Baazizi
* Affiliation: LIP6 - Faculté des Sciences - Sorbonne Université
* Email: mohamed-amine.baazizi@lip6.fr
* October 2023


# Querying nested data

This lab session complements the previous session with several concepts:
* manipulating irregular, nested data represented in JSON format
* using windowing functions, multi-dimensional aggregations, and pivot tables

## Pre-requisite

In [1]:
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

local = "local[*]"
appName = "Spark-Dataframe-JSON"
localConfig = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")

spark = SparkSession.builder.config(conf = localConfig).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

### Data loading

In [3]:
! mkdir -p /tmp/spark-dataframe/

In [4]:
! wget https://nuage.lip6.fr/s/3CFYf9qFNitKKER/download/vk_001.json.zip -O /tmp/spark-dataframe/vk_001.json.zip

--2023-10-09 17:38:13--  https://nuage.lip6.fr/s/3CFYf9qFNitKKER/download/vk_001.json.zip
Resolving nuage.lip6.fr (nuage.lip6.fr)... 132.227.201.11
Connecting to nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15376718 (15M) [application/zip]
Saving to: ‘/tmp/spark-dataframe/vk_001.json.zip’

/tmp/spark-datafram 100%[===================>]  14.66M  5.06MB/s    in 2.9s    

2023-10-09 17:38:18 (5.06 MB/s) - ‘/tmp/spark-dataframe/vk_001.json.zip’ saved [15376718/15376718]



In [5]:
! unzip /tmp/spark-dataframe/vk_001.json.zip -d /tmp/spark-dataframe/

Archive:  /tmp/spark-dataframe/vk_001.json.zip
  inflating: /tmp/spark-dataframe/vk_001.json  


In [6]:
path = '/tmp/spark-dataframe/'

## Dataset loading
The dataset used in this lab session describes posts exchanged in a Russian social network, called VK, during the 2018 Russian elections.
The schema of this dataset is described in the official API website
https://vk.com/dev/streaming_api_docs_2?f=7.%2BЧтение%2Bпотока


We will use this dataset to perform some analytics on the type of posts, the tags used in these post and the relationships between authors of the posts.

In [7]:
data = spark.read.format("json").load(path+'/vk_001.json').distinct()
data.count()

30683

In [8]:
#examine the schema
data.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- code: long (nullable = true)
 |-- event: struct (nullable = true)
 |    |-- action: string (nullable = true)
 |    |-- attachments: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- album: struct (nullable = true)
 |    |    |    |    |-- created: long (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- id: string (nullable = true)
 |    |    |    |    |-- owner_id: long (nullable = true)
 |    |    |    |    |-- size: long (nullable = true)
 |    |    |    |    |-- thumb: struct (nullable = true)
 |    |    |    |    |    |-- access_key: string (nullable = true)
 |    |    |    |    |    |-- album_id: long (nullable = true)
 |    |    |    |    |    |-- date: long (nullable = true)
 |    |    |    |    |    |-- height: long (nullable = true)
 |    |    |    |    |    |-- id: long (nullable = true)
 | 

## Questions

The schema indicates each attribute with nullable=true which is not very informative since we can not know whether a field has missing values.
The following instructions count the number of rows w/o missing values for some specific fields of interest.

In [9]:
att_list = ["event",
                 "event.event_id",
                 "event.event_id.post_id",
                 "event.event_id.post_owner_id",
                 "event.event_id.comment_id",
                 "event.event_id.shared_post_id",
                 "event.author",
                 "event.attachments",
                 "event.geo",
                 "event.tags",
                 "event.creation_time"]

In [10]:
for att in att_list:
    print("count of %s=%s"%(att, data.where(att+ " is not null").count()))

count of event=30683
count of event.event_id=30683
count of event.event_id.post_id=30683
count of event.event_id.post_owner_id=30683
count of event.event_id.comment_id=16518
count of event.event_id.shared_post_id=638
count of event.author=30683
count of event.attachments=15944
count of event.geo=22
count of event.tags=30683
count of event.creation_time=30683


What do you observe? Which attributes have missing values?

event.event_id.comment_id, event.event_id.shared_post_id, event.attachments and event.geo


Feel free to examine the content of these attributes (count the number of distinct values, examine a sample of distinct values, etc)

The official documentation (https://vk.com/dev/streaming_api_docs_2?f=7.%2BЧтение%2Bпотока) reports on some assumptions about the presence/absence of values.
For example, the field `event.event_id.comment_id` is present only when `event.event_type='comment'` and `event.event_id.shared_post_id` is present only when `event.event_type='share'`.

Write two queries to verify these assumptions.

#### Check that `event.event_id.comment_id` is present only when `event.event_type='comment'`

In [11]:
data.filter(data.event.event_id.comment_id.isNotNull() & (data.event.event_type !='comment')).count() == 0

True

#### Check that `event.event_id.shared_post_id` is present only when `event.event_type='share'`

In [12]:
data.filter(data.event.event_id.shared_post_id.isNotNull() & (data.event.event_type !='share')).count() == 0

True

#### How many distinct post ids are there?

In [13]:
from pyspark.sql.functions import count, countDistinct

In [14]:
nb_posts = data.agg(countDistinct(data.event.event_id.post_id).alias('distinct'))
nb_posts.show()

+--------+
|distinct|
+--------+
|   21683|
+--------+



#### How many distinct post_ids per event type

In [15]:
post_per_type = data.groupBy(data.event.event_type).agg(countDistinct(data.event.event_id.post_id).alias('distinct'))
post_per_type.show()

+-----------------+--------+
|event[event_type]|distinct|
+-----------------+--------+
|          comment|   14202|
|             post|    8137|
|            share|     544|
+-----------------+--------+



#### Flattening lists of tags

In `data`, each object is associated with an array of tags accessed from `event.tags` (see the schema). Write an instruction to add a `tag` column containing a single tag obtained by flattening the `tags` array

In [16]:
from pyspark.sql.functions import explode

In [17]:
data_with_tags = data.withColumn("tag", explode(data.event.tags))
data_with_tags.show()

+--------------------+----+--------------------+---------+
|                 _id|code|               event|      tag|
+--------------------+----+--------------------+---------+
|{5a66276e7f254c35...| 100|{new, NULL, {http...| grudinin|
|{5a66296f7f254c35...| 100|{new, NULL, {http...|    putin|
|{5a68d75a713e4d08...| 100|{new, NULL, {http...|    putin|
|{5a68d75a713e4d08...| 100|{new, NULL, {http...|yavlinsky|
|{5a68f01c713e4d08...| 100|{new, NULL, {http...|  navalny|
|{5a68fe3c713e4d08...| 100|{new, NULL, {http...|    putin|
|{5a690364713e4d08...| 100|{new, NULL, {http...|    putin|
|{5a69a403713e4d08...| 100|{new, [{NULL, NUL...|    putin|
|{5a69b263713e4d08...| 100|{new, [{NULL, NUL...|  navalny|
|{5a6a03f7713e4d08...| 100|{new, [{NULL, NUL...|    putin|
|{5a6af994713e4d08...| 100|{new, [{NULL, NUL...|    putin|
|{5a6b13e4713e4d08...| 100|{new, NULL, {http...|    putin|
|{5a6b6900713e4d08...| 100|{new, [{NULL, NUL...|    putin|
|{5a6b7918713e4d08...| 100|{new, [{NULL, NUL...|  navaln

#### Return the number of distinct post_id per tag. Sort in descending order of count

In [18]:
from pyspark.sql.functions import desc
objects_per_tag = data_with_tags.groupBy("tag").agg(countDistinct("event.event_id.post_id").alias("nb_post")).orderBy(desc("nb_post"))

objects_per_tag.show()

+-----------+-------+
|        tag|nb_post|
+-----------+-------+
|      putin|  14859|
|   grudinin|   6222|
|    navalny|   2616|
|    sobchak|   2134|
|zhirinovsky|   1231|
|      titov|    577|
|  yavlinsky|    361|
+-----------+-------+



#### Return the number of distinct author.id per tag. Sort in descending order of count

In [19]:
authors_per_tag = data_with_tags.groupBy("tag").agg(countDistinct("event.author.id").alias("nb_author")).orderBy(desc("nb_author"))

authors_per_tag.show()

+-----------+---------+
|        tag|nb_author|
+-----------+---------+
|      putin|    15673|
|   grudinin|     6207|
|    navalny|     2580|
|    sobchak|     2288|
|zhirinovsky|     1214|
|      titov|      572|
|  yavlinsky|      347|
+-----------+---------+



#### Fact-checking using Wikipedia

Observe that each tag corresponds to a candidate of the RU2018 Elections (Putin, Titov, etc).
We would like to check the relationship between the count of tags per author and the number of votes associated to each candidate.
We collect, from Wikipedia, the number of votes per candidates and make it available though the dataset `votes` defined as follows.

In [20]:
from pyspark.sql.types import *

schema_votes = StructType([StructField("name", StringType()),\
                     StructField("party", StringType()),\
                     StructField("votes", LongType())])


raw_votes = [("putin", "Independent", 56430712),\
             ("grudinin", "Communist",8659206), \
             ("zhirinovsky","Liberal Democratic Party",4154985),\
             ("sobchak","Civic Initiative",1238031),\
             ("yavlinsky","Yabloko",769644), \
             ("titov","Party of Growth",556801)\
            ]

votes = spark.createDataFrame(raw_votes,schema_votes)
votes.show()

+-----------+--------------------+--------+
|       name|               party|   votes|
+-----------+--------------------+--------+
|      putin|         Independent|56430712|
|   grudinin|           Communist| 8659206|
|zhirinovsky|Liberal Democrati...| 4154985|
|    sobchak|    Civic Initiative| 1238031|
|  yavlinsky|             Yabloko|  769644|
|      titov|     Party of Growth|  556801|
+-----------+--------------------+--------+



#### Return for each candidate the number of its votes and the number of authors who tagged it. What do you observe?

In [21]:
votes_count = votes.join(authors_per_tag).where("tag=name").drop("party", "tag")
votes_count.show()

+-----------+--------+---------+
|       name|   votes|nb_author|
+-----------+--------+---------+
|      putin|56430712|    15673|
|   grudinin| 8659206|     6207|
|zhirinovsky| 4154985|     1214|
|    sobchak| 1238031|     2288|
|  yavlinsky|  769644|      347|
|      titov|  556801|      572|
+-----------+--------+---------+



### Window function

Read about the windowing functions https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html.

In [22]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank


Define a window specification for the `votes` and `nb_authors` columns

In [23]:
windowSpecVotes = Window .partitionBy(votes_count['nb_author']).orderBy(votes_count['nb_author'].desc())
windowSpecCount = Window .partitionBy(votes_count['nb_author']).orderBy(votes_count['nb_author'].desc())


In [24]:
windowSpecVotes = Window .orderBy(votes_count['nb_author'].desc())
windowSpecCount = Window .orderBy(votes_count['nb_author'].desc())

Using the window specifications, augment `votes_count` with two attributes, `votes_rank` and `nbAuths_ranks`, which rank each candidate  based on the number of votes he received  and the number of authors who tagged a post with his name.

Hint. The window spans the entire data and does not use any partitioning.

In [25]:
compare = votes_count\
        .withColumn("votes_rank", rank().over(windowSpecVotes))\
        .withColumn("nb_authors_rank", rank().over(windowSpecCount))\

compare.show()


+-----------+--------+---------+----------+---------------+
|       name|   votes|nb_author|votes_rank|nb_authors_rank|
+-----------+--------+---------+----------+---------------+
|      putin|56430712|    15673|         1|              1|
|   grudinin| 8659206|     6207|         2|              2|
|    sobchak| 1238031|     2288|         3|              3|
|zhirinovsky| 4154985|     1214|         4|              4|
|      titov|  556801|      572|         5|              5|
|  yavlinsky|  769644|      347|         6|              6|
+-----------+--------+---------+----------+---------------+



### Multidimensional aggregation

We want to create a cube with three dimensions: `tag`, `event_type` and `month`. While the first two attributes are already available, the month column must be extracted from the `creation_time` attribute using a built-in function.

Create a dataset `dataTagMon` obtained by augmenting `data_with_tags` with an attribute `month` containing the month extracted from the `event.creation_time` attribute.

Hint. You will notice that only months 1, 2 and 3 exist in the dataset.

In [26]:
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import month

In [27]:

data_tag_mon = data_with_tags.withColumn("month", month(from_unixtime(data["event.creation_time"])))

data_tag_mon.select("month").distinct().show()

+-----+
|month|
+-----+
|    1|
|    3|
|    2|
+-----+



#### For each combination of event_type, tag and month, count the nomber of distinct post_ids

In [28]:
from pyspark.sql.functions import approx_count_distinct
from pyspark.sql.functions import col

tag_event_month = data_tag_mon.withColumn("event_type", col("event.event_type"))
tag_event_month = tag_event_month.withColumn("post_id", col("event.event_id.post_id"))
tag_event_month = tag_event_month.select("event_type", "tag", "month", 'post_id')
tag_event_month = tag_event_month.groupBy("event_type", "tag", "month").agg(countDistinct("post_id").alias("count_dist_postids"))

tag_event_month.orderBy(desc("count_dist_postids")).show()

+----------+--------+-----+------------------+
|event_type|     tag|month|count_dist_postids|
+----------+--------+-----+------------------+
|   comment|   putin|    2|              4191|
|   comment|   putin|    3|              3543|
|      post|   putin|    2|              2963|
|      post|   putin|    3|              2954|
|   comment|grudinin|    2|              2012|
|   comment|grudinin|    3|              1484|
|      post|grudinin|    2|              1409|
|   comment|   putin|    1|              1390|
|      post|grudinin|    3|              1206|
|      post|   putin|    1|              1104|
|   comment| navalny|    2|               780|
|   comment| sobchak|    3|               538|
|      post| sobchak|    2|               503|
|   comment| navalny|    1|               474|
|      post| navalny|    2|               461|
|   comment|grudinin|    1|               442|
|   comment| sobchak|    2|               436|
|      post| sobchak|    3|               425|
|   comment| 

### Pivot table

#### Using the `tag_event_month` table, create a pivot table by reducing the dimensions to month and event type.

In [29]:
from pyspark.sql import functions as F
month_event_type = (tag_event_month.groupBy("month").pivot("event_type").agg(F.sum("count_dist_postids"))).orderBy('month')
month_event_type.printSchema()
month_event_type.show()

root
 |-- month: integer (nullable = true)
 |-- comment: long (nullable = true)
 |-- post: long (nullable = true)
 |-- share: long (nullable = true)

+-----+-------+----+-----+
|month|comment|post|share|
+-----+-------+----+-----+
|    1|   2564|1986|  100|
|    2|   7873|5863|  372|
|    3|   6424|5328|  207|
+-----+-------+----+-----+



###  Tag co-occurrence matrix

#### create a dataframe indicating for each pair of distinct tags the author that uses both of them.

In [48]:
auth_tag = data_with_tags.select("tag","event.author.id")
tag_co_occur = auth_tag.alias('a').join(auth_tag.alias('b'),['id'])
tag_co_occur = tag_co_occur.select(col("a.id"), col('a.tag'), col('b.tag').alias('otherTag'))
tag_co_occur = tag_co_occur.filter(col("tag") != col("otherTag"))
tag_co_occur.show()

+---------+-----------+-----------+
|       id|        tag|   otherTag|
+---------+-----------+-----------+
|  2000511|      putin|  yavlinsky|
|  2000511|  yavlinsky|      putin|
|121013525|      putin|   grudinin|
|121013525|      putin|    navalny|
|121013525|      putin|   grudinin|
|121013525|      putin|   grudinin|
|121013525|      putin|   grudinin|
|121013525|      putin|   grudinin|
| 67260949|      putin|    navalny|
|469347470|   grudinin|      putin|
|468277944|   grudinin|zhirinovsky|
| 54818744|   grudinin|    navalny|
| 54818744|   grudinin|zhirinovsky|
| 54818744|zhirinovsky|    navalny|
| 54818744|zhirinovsky|   grudinin|
| 54818744|zhirinovsky|   grudinin|
|329909622|   grudinin|      putin|
|329909622|   grudinin|    navalny|
|329909622|    navalny|      putin|
|329909622|    navalny|   grudinin|
+---------+-----------+-----------+
only showing top 20 rows



In [49]:
# équivalent à l'autre façon juste pour essayer
auth_tag = data_with_tags.select("tag","event.author.id")
tag_co_occur = auth_tag.alias('a').join(auth_tag.alias('b'), ((col('a.id') == col('b.id')) & (col('b.tag')!=col('b.tag'))))
tag_co_occur = tag_co_occur.select(col("a.id"), col('a.tag'), col('b.tag').alias('otherTag'))
tag_co_occur.show()

+---------+-----------+-----------+
|       id|        tag|   otherTag|
+---------+-----------+-----------+
|  2000511|      putin|  yavlinsky|
|  2000511|  yavlinsky|      putin|
|121013525|      putin|   grudinin|
|121013525|      putin|    navalny|
|121013525|      putin|   grudinin|
|121013525|      putin|   grudinin|
|121013525|      putin|   grudinin|
|121013525|      putin|   grudinin|
| 67260949|      putin|    navalny|
|469347470|   grudinin|      putin|
|468277944|   grudinin|zhirinovsky|
| 54818744|   grudinin|    navalny|
| 54818744|   grudinin|zhirinovsky|
| 54818744|zhirinovsky|    navalny|
| 54818744|zhirinovsky|   grudinin|
| 54818744|zhirinovsky|   grudinin|
|329909622|   grudinin|      putin|
|329909622|   grudinin|    navalny|
|329909622|    navalny|      putin|
|329909622|    navalny|   grudinin|
+---------+-----------+-----------+
only showing top 20 rows



#### Build the tag co-occurence matrix indicating for each pair of tags the number of authors that use them

In [50]:
co_occur_mat = tag_co_occur.groupBy('Tag').pivot('otherTag').count()

co_occur_mat.show()

+-----------+--------+-------+-----+-------+-----+---------+-----------+
|        Tag|grudinin|navalny|putin|sobchak|titov|yavlinsky|zhirinovsky|
+-----------+--------+-------+-----+-------+-----+---------+-----------+
|  yavlinsky|     210|     60|  355|    199|   94|     NULL|        145|
|   grudinin|    NULL|   1113| 7267|    722|  140|      210|        714|
|      titov|     140|     21|  246|    111| NULL|       94|         90|
|    navalny|    1113|   NULL| 1806|    300|   21|       60|         90|
|    sobchak|     722|    300| 1299|   NULL|  111|      199|        439|
|zhirinovsky|     714|     90|  901|    439|   90|      145|       NULL|
|      putin|    7267|   1806| NULL|   1299|  246|      355|        901|
+-----------+--------+-------+-----+-------+-----+---------+-----------+



## END